# Statistics for Experimental Physics
## Part II: Bayesian Statistics
### Assessed Problem Sheet 4

**Due: 11 December 2024**  
**Boris Leistedt** - B.LEISTEDT@IMPERIAL.AC.UK

Based on the material originally designed by Alan Heavens

Hand in by 5 pm Thursday 11th December. 

**Please submit a project-like report (PDF) with the results, and a copy of your code included. There are no page limits.**


## 1. Cosmology with Supernovae Ia

In this exercise, you will write **your own HMC code** to infer cosmological parameters from supernova Ia data. Do not use a package for the main HMC implementation, but you may use packages (e.g. corner.py) to display results.

**Objectives:**
1. Implement the cosmological distance-redshift relation for a flat universe
2. Build a Hamiltonian Monte Carlo sampler from scratch
3. Infer $\Omega_m$ (matter density) and $h$ (Hubble constant) from real supernova data (Pantheon+ dataset: 1701 supernovae)
4. Assess convergence using trace plots, autocorrelation, and effective sample size
5. Explore how HMC parameters affect sampling efficiency

**The Big Picture:** Type Ia supernovae are "standard candles" with known intrinsic brightness. By measuring their apparent brightness at different redshifts, we can map the distance-redshift relation, which depends on the Universe's composition ($\Omega_m$, $\Omega_v$, $h$). This same technique led to the 1998 Nobel Prize discovery that the Universe's expansion is accelerating, revealing the existence of dark energy.

### 1.1. Background and Theory

#### Type Ia Supernovae: Standard Candles for Cosmology

**Type Ia supernovae** are thermonuclear explosions of white dwarfs that reach the Chandrasekhar limit (~1.4 solar masses), making them excellent **standard candles**:
- Nearly uniform peak luminosity ($L \approx 10^{43}$ erg/s) after empirical corrections
- Visible to cosmological distances (billions of light-years)
- Led to the 1998 Nobel Prize discovery of accelerating cosmic expansion (Perlmutter, Schmidt, Riess)

#### Cosmological Framework

The Universe is expanding, causing **cosmological redshift**:
$$z \equiv \frac{\lambda_{\text{observed}} - \lambda_{\text{emitted}}}{\lambda_{\text{emitted}}} = \frac{a(t_{\text{now}})}{a(t_{\text{emission}})} - 1$$

where $a(t)$ is the scale factor ($a=1$ today). 

**Our goal is to measure the expansion rate and composition of the universe** by observing supernovae at different redshifts. Because supernovae are standard candles with known intrinsic brightness, measuring their apparent brightness (magnitude) tells us their distance. By mapping how distance varies with redshift for many supernovae, we can reconstruct the expansion history, which is governed by the cosmological parameters $\Omega_m$ and $h$. Different values of these parameters produce different distance-redshift curves—this is what allows us to infer cosmology from supernova observations.

The expansion is governed by the **Friedmann equation**:
$$H(z)^2 = H_0^2\left[\Omega_m(1+z)^3 + \Omega_k(1+z)^2 + \Omega_v\right]$$

**Parameters:**
- $H_0 = 100h$ km/s/Mpc: Hubble constant ($h \approx 0.7$, typical range 0.6-0.8)
- $\Omega_m \approx 0.3$: matter density parameter (typical range 0.2-0.4)
- $\Omega_v \approx 0.7$: dark energy density parameter
- $\Omega_k = 1 - \Omega_m - \Omega_v$: curvature parameter

We assume a **flat universe** ($\Omega_m + \Omega_v = 1$, $\Omega_k = 0$), well-supported by CMB observations.

#### Distance Measurements

The flux from a supernova of luminosity $L$ at **luminosity distance** $D_L$ is:
$$f = \frac{L}{4\pi D_L^2}$$

For a flat universe, $D_L$ is given by:
$$D_L(z) = 3000h^{-1}(1 + z) \int_0^z \frac{dz'}{\sqrt{\Omega_m(1 + z')^3 + 1 - \Omega_m}} \text{ Mpc}$$

Since HMC requires millions of likelihood evaluations, we use the **Pen (1999) analytical approximation** (accurate to <0.4%):
$$D_L(z; \Omega_m, h) = \frac{c}{H_0}(1 + z)\left[\eta(1, \Omega_m) - \eta\left(\frac{1}{1 + z}, \Omega_m\right)\right]$$

where $c = 299792.458$ km/s and:
$$\eta(a, \Omega_m) = 2\sqrt{s^3 + 1}\left[\frac{1}{a^4} - 0.1540\frac{s}{a^3} + 0.4304\frac{s^2}{a^2} + 0.19097\frac{s^3}{a} + 0.066941s^4\right]^{-1/8}$$

with $s^3 = (1 - \Omega_m)/\Omega_m$ and $a = 1/(1+z)$ (valid for $0.2 \leq \Omega_m \leq 1$).

Astronomers measure brightness using **magnitudes** ($m = -2.5\log_{10}f + \text{const}$). The **distance modulus** is:
$$\mu = m - M = 5\log_{10}\left(\frac{D_L}{\text{Mpc}}\right) + 25$$

Factoring out $h$ using $D_L^* \equiv D_L(h=1)$:
$$\mu(z; \Omega_m, h) = 25 - 5\log_{10} h + 5\log_{10}\left(\frac{D_L^*}{\text{Mpc}}\right)$$

where $h$ produces a vertical shift and $\Omega_m$ affects the curve shape.



### 1.2. Data

The data file (from the 'Pantheon+' sample - see https://arxiv.org/abs/2112.03863 for more detail) consists of data from 1701 supernovae, with a redshift and distance modulus $\mu$ for each supernova. 

The file `Pantheon+SH0ES.dat` (from Teams or Blackboard) contains the data:
- `zHD`: redshift you should use
- `MU_SH0ES`: distance modulus

The covariance matrix $C$ is provided in `Pantheon+SH0ES_STAT+SYS.cov.txt` (from Teams or Blackboard).
It is a square symmetric matrix N x N.
The file provided contains a vector (the first row is the size N) which you can just reshape to N x N.

The files can also be downloaded here:
- https://www.dropbox.com/scl/fi/n67of2kwtmabb2vahk36m/Pantheon-SH0ES.dat?rlkey=mw210zbna7b0pxs4ptj2gdivl&dl=0
- https://www.dropbox.com/scl/fi/ncafbjkwh5w83hehs6p0u/Pantheon-SH0ES_STAT-SYS.cov.txt.zip?rlkey=ruxgi74dkz1hcfehxdl4jfns9&dl=0 

### 1.3. Exercise

Write your own HMC code to infer $h$ and $\Omega_m$ from the supernova dataset, assuming the Universe is flat and the errors are Gaussian.

**Requirements:**

- Nearby supernovae have redshifts significantly altered by 'peculiar velocities', not associated with the general expansion of the Universe. **Discard supernovae with $z < 0.01$.**
- $h$ and $\Omega_m$ are positive, and have values of the rough order of unity
- Assume uniform priors on the parameters (so you will sample from the likelihood)
- Explore visually (with trace plots) the chain, and compute and report the acceptance rate
- Choose a suitable burn-in and say why you chose it
- Compute the average value of the parameters under the posterior distribution, and their variances and covariance
- Compute and display the correlation function of the chain. Compute the acceptance rate and the effective number of samples, using the formula in the lectures. Explore how these changes as the parameters of the HMC are varied

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
torch.set_default_dtype(torch.float64)
from pathlib import Path

# Add your imports here

#### Load and prepare the data

In [6]:
# Load supernova data
# Filter z < 0.01
# Your code here

DATA_DIR = Path(".")
SN_FILE = DATA_DIR/"Pantheon+SH0ES.dat"
COV_FILE = DATA_DIR / "Pantheon+SH0ES_STAT+SYS.cov.txt"

df = pd.read_csv(SN_FILE, delim_whitespace=True)
expected_columns = {"zHD", "MU_SH0ES"}
print(set(df.columns).issuperset(expected_columns))

# Discard nearby SNe with z < 0.01, preserve original ordering
mask = df["zHD"].to_numpy() >= 0.01
idx = np.where(mask)[0]

df_f = df.loc[mask].reset_index(drop=True)
z_data = df_f["zHD"].to_numpy(dtype=np.float64)
mu_data = df_f["MU_SH0ES"].to_numpy(dtype=np.float64)

print(f"Loaded {len(df)} SNe; kept {len(z_data)} with z >= 0.01")
print(f"Data columns: {list(df.columns)}")

True
Loaded 1701 SNe; kept 1590 with z >= 0.01
Data columns: ['CID', 'IDSURVEY', 'zHD', 'zHDERR', 'zCMB', 'zCMBERR', 'zHEL', 'zHELERR', 'm_b_corr', 'm_b_corr_err_DIAG', 'MU_SH0ES', 'MU_SH0ES_ERR_DIAG', 'CEPH_DIST', 'IS_CALIBRATOR', 'USED_IN_SH0ES_HF', 'c', 'cERR', 'x1', 'x1ERR', 'mB', 'mBERR', 'x0', 'x0ERR', 'COV_x1_c', 'COV_x1_x0', 'COV_c_x0', 'RA', 'DEC', 'HOST_RA', 'HOST_DEC', 'HOST_ANGSEP', 'VPEC', 'VPECERR', 'MWEBV', 'HOST_LOGMASS', 'HOST_LOGMASS_ERR', 'PKMJD', 'PKMJDERR', 'NDOF', 'FITCHI2', 'FITPROB', 'm_b_corr_err_RAW', 'm_b_corr_err_VPEC', 'biasCor_m_b', 'biasCorErr_m_b', 'biasCor_m_b_COVSCALE', 'biasCor_m_b_COVADD']


C:\Users\Mihir Koka\AppData\Local\Temp\ipykernel_700\3155238785.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(SN_FILE, delim_whitespace=True)


In [ ]:
# Load covariance matrix and extract corresponding subset after filtering
# Your code here

#### Implement the theoretical model (Pen 1999 formula)

In [ ]:
def eta(a, Omega_m):
    """
    Helper function for Pen (1999) formula.
    
    Parameters:
    -----------
    a : float or array
        Scale factor (a = 1/(1+z))
    Omega_m : float
        Matter density parameter
    
    Returns:
    --------
    eta : float or array
    """
    # Your implementation here
    pass

def luminosity_distance_star(z, Omega_m):
    """
    Calculate D_L^* (luminosity distance with h=1 factored out) using Pen (1999).
    
    Parameters:
    -----------
    z : float or array
        Redshift
    Omega_m : float
        Matter density parameter
    
    Returns:
    --------
    D_L_star : float or array
        Luminosity distance in Mpc (with h=1)
    """
    # Your implementation here
    pass

def distance_modulus(z, Omega_m, h):
    """
    Calculate theoretical distance modulus.
    
    Parameters:
    -----------
    z : float or array
        Redshift
    Omega_m : float
        Matter density parameter
    h : float
        Reduced Hubble constant
    
    Returns:
    --------
    mu : float or array
        Distance modulus
    """
    # Your implementation here
    pass


# Test these functions! Check that you can get reasonable values given typical parameters and redshifts. 

#### Implement the likelihood and its gradient

In [ ]:
def log_likelihood(params, z_data, mu_data, C_inv):
    """
    Calculate log-likelihood for supernova data.
    
    Parameters:
    -----------
    params : array
        [Omega_m, h]
    z_data : array
        Redshift data
    mu_data : array
        Distance modulus data
    C_inv : array
        Inverse covariance matrix
    
    Returns:
    --------
    log_L : float
        Log-likelihood value
    """
    # Your implementation here
    pass

def gradient_log_likelihood(params, z_data, mu_data, C_inv):
    """
    Calculate gradient of log-likelihood with respect to parameters.
    
    Three options: 
    - derive the gradient analytically and implement it
    - use numerical differentiation (finite differences)
    - use automatic differentiation
    
    Parameters:
    -----------
    params : array
        [Omega_m, h]
    z_data : array
        Redshift data
    mu_data : array
        Distance modulus data
    C_inv : array
        Inverse covariance matrix
    
    Returns:
    --------
    grad : array
        Gradient [d(log L)/d(Omega_m), d(log L)/dh]
    """
    # Your implementation here
    pass

#### Implement HMC sampler

Implement the Hamiltonian Monte Carlo algorithm, including:
- Leapfrog integrator for Hamiltonian dynamics
- Metropolis acceptance step
- Main HMC loop

In [ ]:
def leapfrog(q, p, epsilon, L, grad_log_prob, *args):
    """
    Leapfrog integrator for Hamiltonian dynamics.
    
    Parameters:
    -----------
    q : array
        Current position (parameters)
    p : array
        Current momentum
    epsilon : float
        Step size
    L : int
        Number of leapfrog steps
    grad_log_prob : function
        Function to calculate gradient of log probability
    *args : additional arguments
        Additional arguments to pass to grad_log_prob
    
    Returns:
    --------
    q_new : array
        New position
    p_new : array
        New momentum
    """
    # Your implementation here
    pass

def hmc_sampler(initial_params, n_samples, epsilon, L, log_prob, grad_log_prob, *args):
    """
    Hamiltonian Monte Carlo sampler.
    
    Parameters:
    -----------
    initial_params : array
        Initial parameter values [Omega_m, h]
    n_samples : int
        Number of samples to generate
    epsilon : float
        Step size for leapfrog integrator
    L : int
        Number of leapfrog steps per iteration
    log_prob : function
        Log probability function
    grad_log_prob : function
        Gradient of log probability
    *args : additional arguments
        Additional arguments to pass to log_prob and grad_log_prob
    
    Returns:
    --------
    samples : array
        MCMC samples (n_samples x n_params)
    acceptance_rate : float
        Fraction of proposals accepted
    """
    # Your implementation here
    pass

#### Run HMC sampler

In [ ]:
# Set HMC parameters
# initial_params = [Omega_m_init, h_init]
# n_samples = ...
# epsilon = ...
# L = ...

# Run HMC
# samples, acceptance_rate = hmc_sampler(...)

# Your code here

#### Trace plots and acceptance rate

Create trace plots to visualize the MCMC chain and report the acceptance rate.

In [ ]:
# Create trace plots
# Your code here

#### Burn-in and posterior analysis

Choose an appropriate burn-in period and compute posterior statistics.

In [ ]:
# Choose burn-in based on trace plots
# burn_in = ...

# Remove burn-in samples
# samples_post_burnin = samples[burn_in:]

# Compute posterior mean
# mean_Omega_m = ...
# mean_h = ...

# Compute posterior variance
# var_Omega_m = ...
# var_h = ...

# Compute posterior covariance
# cov_matrix = ...

# Your code here

#### Autocorrelation function and effective sample size

Compute and plot the autocorrelation function, then calculate the effective number of independent samples.

In [ ]:
# Compute autocorrelation function
# Your code here

# Plot autocorrelation
# Your code here

# Compute effective sample size
# Use formula from lectures: N_eff = N / (1 + 2*sum(rho_k))
# Your code here

#### Exploration of HMC parameter effects

Systematically explore how the step size $\epsilon$ and number of leapfrog steps $L$ affect:
- Acceptance rate
- Effective sample size
- Computational efficiency

In [ ]:
# Vary epsilon and L systematically
# For each combination, run HMC and compute acceptance rate and N_eff
# Your code here

#### Visualization of results

Create visualizations of the posterior distribution.

In [ ]:
# Create 2D posterior plot (contours or scatter)
# Can use corner.py package: corner.corner(samples_post_burnin, labels=[r'$\Omega_m$', r'$h$'])
# Your code here

### 1.4. Optional Extension

- Write and apply a Gelman-Rubin convergence test, and deduce roughly how long the chains should be for convergence.

In [ ]:
# Optional: Gelman-Rubin convergence test
# Run multiple chains from different starting points
# Compute R-hat statistic
# Your code here